# Set up

In [ ]:
import tensorflow as tf
import os
from keras.utils import img_to_array, load_img
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

trainPath = r"C:\Users\gener\OneDrive\Documents\fruits360\fruits-360\Train"
testPath = r"C:\Users\gener\OneDrive\Documents\fruits360\fruits-360\Test"

batchSize = 64      # Reduce value if you have less GPU


# Create models

In [ ]:
def model1(numClasses):
    """
    This model is from the Eran guy's tutorial
    """
    model = Sequential()
    # Base
    model.add(Conv2D(filters=128, kernel_size=3, activation="relu", input_shape=(100,100,3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(filters=64, kernel_size=3, activation="relu"))
    model.add(Conv2D(filters=32, kernel_size=3, activation="relu"))
    model.add(MaxPooling2D())
    model.add(Dropout(0.5))
    model.add(Flatten())
    # Head
    model.add(Dense(5000, activation="relu"))
    model.add(Dense(1000, activation="relu"))
    model.add(Dense(numClasses, activation="softmax"))
    return model

def model2(numClasses):
    """
    Tries reorganizing and standardizing convolutional blocks based on Kaggle lesson
    """
    model = Sequential()
    # Base
    model.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(100,100,3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D())
    # Head
    model.add(Flatten())
    model.add(Dense(5000, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation="relu"))
    model.add(Dense(numClasses, activation="softmax"))     # CHANGE THE 131
    return model

def model3():
    """
    Uses depthwise convolutional blocks
    """
    from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, GlobalAveragePooling2D, Dense, Reshape, Activation, Add

def conv_block(inputs, filters, kernel_size, strides):
    model = Sequential()
    model.add(Conv2D(filters, kernel_size, strides=strides, padding='same'))
    model.add(Activation('relu'))
    return model(inputs)

def depthwise_conv_block(inputs, kernel_size, strides):
    model = Sequential()
    model.add(DepthwiseConv2D(kernel_size, strides=strides, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(256, (1, 1), padding='same'))
    model.add(Activation('relu'))
    return model(inputs)

def mobileNetV3():
    model = Sequential()
    
    model.add(Conv2D(16, (3, 3), strides=(2, 2), padding='same', input_shape=(224, 224, 3)))
    
    model.add(conv_block(32, (3, 3), strides=(2, 2)))
    model.add(depthwise_conv_block((3, 3), strides=(1, 1)))
    
    model.add(conv_block(64, (3, 3), strides=(2, 2)))
    model.add(depthwise_conv_block((3, 3), strides=(1, 1)))
    model.add(depthwise_conv_block((3, 3), strides=(1, 1)))
    model.add(depthwise_conv_block((3, 3), strides=(1, 1)))
    model.add(depthwise_conv_block((3, 3), strides=(1, 1)))
    model.add(depthwise_conv_block((3, 3), strides=(1, 1)))

    model.add(conv_block(128, (3, 3), strides=(2, 2)))
    model.add(depthwise_conv_block((3, 3), strides=(1, 1)))
    model.add(depthwise_conv_block((3, 3), strides=(1, 1)))
    model.add(depthwise_conv_block((3, 3), strides=(1, 1)))

    model.add(conv_block(256, (3, 3), strides=(2, 2)))
    model.add(depthwise_conv_block((3, 3), strides=(1, 1)))

    model.add(GlobalAveragePooling2D())
    model.add(Reshape((1, 1, 256)))
    
    model.add(Conv2D(1280, (1, 1), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(num_classes, (1, 1), padding='same'))
    model.add(Activation('softmax'))
    
    return model

# Create the MobileNetV3 model
model = mobileNetV3()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



def model4(numClasses):
    """
    Use transfer learning with MobileNetV3 model.
    """

def model5(numClasses):
    """
    Try preprocessing layers 
    """

def model6(numClasses):
    """
    Two output layers for fruits and vegetables
    """



